In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sporgboost.preprocessing import onehot_encode, shuffle
from sporgboost.trees import AxisAlignedDecisionTree, SparseRandomDecisionTree as SPODT
from sklearn.tree import DecisionTreeClassifier
sktree = DecisionTreeClassifier()
model = SPODT(d = 2, s = 3)

In [2]:
# Dataset for testing
X, y = load_iris(return_X_y = True)

# Set seed to be able to reproduce
np.random.seed(1234)
    
# Preprocessing
y = onehot_encode(y)
X, y = shuffle(X, y)

In [201]:
from sporgboost.projections import identity, sparse_random
from numba import njit, prange, deferred_type, optional, uint32, float64
from numba.experimental import jitclass
from sporgboost.common import best_split, gini_impurity
from sporgboost.utils import row_mean

node_type = deferred_type()

# Node needs to be explicitly included in each tree type for numba
# to properly compile
@jitclass([
    ('value', optional(float64[:,:])),
    ('left', optional(node_type)),
    ('right', optional(node_type)),
    ('proj', optional(float64[:,:])),
    ('split', optional(float64))
])
class Node():
    def __init__(self):
        self.value = None
        self.left = None
        self.right = None
        self.proj = None
        self.split = None

    def is_leaf(self):
        return self.left is None and self.right is None

    def init_children(self):
        self.left = Node()
        self.right = Node()
        
node_type.define(Node.class_type.instance_type)

# @njit(parallel = False)
def fit(X, y, max_depth = 8):
    # Initialize root of tree
    root = Node()

    # Each piece of work contains a pointer to 
    # the node being processed and the index positions
    # of obs at that node
    nodes = [(root, np.arange(0, X.shape[0]))]

    depth = 0
    start = 0
    end = 1
    while (depth < max_depth) and ((end - start) > 0):
        # Parallel loop over all nodes to be processed
        for node_idx in prange(start, end):
            # Get node and asociated obs
            node, idx = nodes[node_idx]
            X_, y_ = X[idx, :], y[idx, :]

            # Step 1: Check if node is a leaf
            node.value = row_mean(y_).reshape((1, -1))
            if gini_impurity(node.value) == 0:
                continue

            # Step 2: If node is not a leaf, find a split
            # Project data based on function
            A = identity(X)
            X_proj = X_ @ A

            # Evaluate each col and candidate split
            col, node.split = best_split(X_proj, y_)
            node.proj = A[col, :].reshape((-1, 1))

            # Initalize children and add to the next iteration to be processed
            node.init_children()

            # Get idx arrays for the split
            le = X_proj[:, col] <= node.split

            nodes.append((node.left, idx[le]))
            nodes.append((node.right, idx[~le]))
        
        # Once new nodes have been added, increment the index and continue adding nodes if needed
        nodes_processed_in_round = end - start
        start += nodes_processed_in_round
        end += nodes_processed_in_round
        depth += 1
    return root

t = fit(X, y)
t

In [202]:
%timeit fit(X, y)

1.57 ms ± 17.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [191]:
%timeit sktree.fit(X, y)

333 µs ± 8.34 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [16]:
%timeit sktree.fit(X, y)

307 µs ± 5.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
%timeit model.predict(X)

c:\Users\Ed\OneDrive - Johns Hopkins\Desktop\Adv_ML_Project\sporgboost\common\_predict.py:13: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  X_ = np.dot(X, tree.proj)


The slowest run took 4.33 times longer than the fastest. This could mean that an intermediate result is being cached.
37.7 µs ± 26.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit sktree.predict(X)

71.3 µs ± 851 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
np.all(model.predict(X) == sktree.predict(X))

c:\Users\Ed\OneDrive - Johns Hopkins\Desktop\Adv_ML_Project\sporgboost\common\_predict.py:13: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  X_ = np.dot(X, tree.proj)


True